In [183]:
import pandas as pd
df = pd.read_csv("movies.csv")

In [184]:
'''import pandas as pd

try:
    answersDataFrane = pd.read_csv("AnkietaOFilmachIMDB.csv", encoding='latin1', sep=';')
    print("File loaded successfully with 'latin1' encoding and comma delimiter.")
except UnicodeDecodeError:
    try:
        answersDataFrane = pd.read_csv("AnkietaOFilmachIMDB.csv", encoding='windows-1252', sep=';')
        print("File loaded successfully with 'windows-1252' encoding and comma delimiter.")
    except Exception as e:
        print(f"An error occurred with encoding or delimiter: {e}")
except FileNotFoundError:
    print("Error: The file 'AnkietaOFilmachIMDB.csv' was not found.")
    '''

'import pandas as pd\n\ntry:\n    answersDataFrane = pd.read_csv("AnkietaOFilmachIMDB.csv", encoding=\'latin1\', sep=\';\')\n    print("File loaded successfully with \'latin1\' encoding and comma delimiter.")\nexcept UnicodeDecodeError:\n    try:\n        answersDataFrane = pd.read_csv("AnkietaOFilmachIMDB.csv", encoding=\'windows-1252\', sep=\';\')\n        print("File loaded successfully with \'windows-1252\' encoding and comma delimiter.")\n    except Exception as e:\n        print(f"An error occurred with encoding or delimiter: {e}")\nexcept FileNotFoundError:\n    print("Error: The file \'AnkietaOFilmachIMDB.csv\' was not found.")\n    '

## Data preparation for user data

In [185]:
#answersDataFrane.replace(-1, pd.NA).describe(include='all')


In [186]:
def rating_cleaner(CurrentDataFrame, rating_name):

    if CurrentDataFrame[rating_name].dtype!="int64":

        if CurrentDataFrame[rating_name].dtype!="float64":
            CurrentDataFrame[rating_name] = CurrentDataFrame[rating_name].astype(str).str.strip()
            CurrentDataFrame[rating_name] = pd.to_numeric(CurrentDataFrame[rating_name], errors='coerce')

        CurrentDataFrame = CurrentDataFrame.fillna(-1)    
        if CurrentDataFrame[rating_name].dtype == 'float64':
            CurrentDataFrame[rating_name] = CurrentDataFrame[rating_name].astype('int64')
    return CurrentDataFrame

In [187]:
# --- User Data preparation ---
answersDataFrane = pd.read_csv("AnkietaOFilmachIMDB.csv", encoding='latin1', sep=';')
for col in answersDataFrane.columns[1:]:
    answersDataFrane = rating_cleaner(answersDataFrane, col)

# --- Merge movie data with user ratings using a common key ---
# First, rename 'whose_rating' to 'name' to match the main df
answersDataFrane.rename(columns={'whose_rating': 'name'}, inplace=True)
# Then, merge the two DataFrames on the 'name' column
df = pd.merge(df, answersDataFrane, on='name', how='left')
for col in df.columns:
    if col.startswith('rating_'):
        df[col] = df[col].fillna(-1)

# --- Data preparation for project ---
# ... (your existing code for one-hot encoding, etc.) ...
#df = one_hot_encode(df, ("genre", "cast_name", "writter_name", "director_name"), prefixes = ("g_", "c_", "w_", "d_") )

#certificate_numbers = ([])
#certificate_labels = df["certificate"].unique()
# ... (your certificate mapping code) ...
#df.replace(mapping, inplace=True)

#df = df.drop(columns="duration")

# *** REMOVE THIS LINE: df = df.dropna() ***

# ... (the rest of your data preparation) ...
# You should handle missing values within test_setups, as it's currently doing.

In [188]:
df = df.sort_values(by="name")

In [189]:
'''for col in answersDataFrane.columns[1:]:
    extracted_col = answersDataFrane[col]
    df = pd.concat([df, extracted_col], axis=1)'''

'for col in answersDataFrane.columns[1:]:\n    extracted_col = answersDataFrane[col]\n    df = pd.concat([df, extracted_col], axis=1)'

## Data preparation for project

### usuniecie czesci kolumn

In [190]:
if "img_link" in df.columns: df.drop(columns="img_link", inplace=True)
if "id" in df.columns: df.drop(columns="id", inplace=True)
if "cast_id" in df.columns: df.drop(columns="cast_id", inplace=True)
if "director_id" in df.columns: df.drop(columns="director_id", inplace=True)
if "writter_id" in df.columns: df.drop(columns="writter_id", inplace=True)

### funkcja do zmiany stringu w listę

In [191]:
def split_if_string(value):
    if isinstance(value, str):
        return value.split(',')
    return value # Return as is if not a string (e.g., already a list or NaN)

In [192]:
df['cast_name'] = df['cast_name'].apply(split_if_string)
df['writter_name'] = df['writter_name'].apply(split_if_string)
df['director_name'] = df['director_name'].apply(split_if_string)
df['genre'] = df['genre'].apply(split_if_string)

In [193]:
df['duration'] = pd.to_numeric(df['duration'], errors='coerce')

In [194]:
bins = [0, 60, 90, 120, 150, 180, float('inf')]
labels = [0, 1, 2, 3, 4, 5]
#labels = ['very short', 'short', 'normal', 'pretty long', 'long', 'very long']


df['duration_category'] = pd.cut(df['duration'], bins=bins, labels=labels, right=False)


In [195]:
df['duration_category'] = pd.to_numeric(df['duration_category'], errors='coerce')

# Przygotowanie pod Sieć neuronową

### usuniecie nazw, oraz rank

In [196]:

if "rank" in df.columns: df.drop(columns="rank", inplace= True)

### function for one-hot encoding

In [197]:
def one_hot_encode (DataFrame, column_names, prefixes):
    dfInWork = DataFrame
    for column_name, prefix in zip(column_names, prefixes):
        if column_name in dfInWork.columns: 
            df_exploded = dfInWork[column_name].explode()
            df_dummies = pd.get_dummies(df_exploded, prefix=prefix)
            df_genres_encoded = df_dummies.groupby(df_dummies.index).max()
            df_final = pd.merge(dfInWork, df_genres_encoded, left_index=True, right_index=True, how='left')
            dfInWork = df_final.drop(column_name, axis=1)
        else:
            print(f"ERROR, column {column_name} non existing")
    return dfInWork


In [198]:
df = one_hot_encode(df, ("genre", "cast_name", "writter_name", "director_name"), prefixes = ("g_", "c_", "w_", "d_") )


In [199]:
certificate_numbers = ([])
certificate_labels = df["certificate"].unique()

i = 0
for unique in certificate_labels:
    certificate_numbers.append(i)
   # certificate_labels.append(unique)
    i+=1

label_to_number_map = dict(zip(certificate_labels, certificate_numbers))

mapping = {
    "certificate": label_to_number_map
}

df.replace(mapping, inplace=True)



C:\Users\kubah\AppData\Local\Temp\ipykernel_28860\3106441032.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(mapping, inplace=True)


In [200]:
df = df.drop(columns="duration")

### pozbycie się nulli

In [201]:
df = df.dropna()

## pozbycie się ciągłości

In [202]:
df["imdb_rating"] *=10

In [203]:

df.replace(mapping, inplace=True)
for i in df.columns:
    if df[i].dtype == 'float64':
        df[i] = df[i].astype('int64')

### pozbycie się rzadkich aktorów

In [204]:
deleted = 0
undeleted = 0

for column in df.columns:
    if df[column].dtype == "bool":
        if df[column].values.sum() < 3:
            df.drop(columns=column, inplace=True)
            deleted += 1
        else:
            undeleted+=1
print(f"undeleted: {undeleted}")
print(f"deleted: {deleted}")

undeleted: 190
deleted: 4277


### discretising year and number of votes

sprawdzenie wartości dla danych lat

In [205]:
bins = [0, 1944, 1960, 1968, 1994, 2007, 2015, float('inf')]
labels = [0, 1, 2, 3, 4, 5, 6]
#labels = ['very short', 'short', 'normal', 'pretty long', 'long', 'very long']


df['year_category'] = pd.cut(df['year'], bins=bins, labels=labels, right=False)

In [206]:
df = df.drop(columns="year")

### to samo, ale z liczbą głosów

od 1740000 są wysokie,
from 1030000 moderate

In [207]:
voting_ranges = [0, 2*10**5, 4*10**5, 7*10**5, 10**6, float('inf')]
labels = [0, 1, 2, 3, 4, 5, 6]
#labels = ['very short', 'short', 'normal', 'pretty long', 'long', 'very long']


df['imbd_votes_category'] = pd.cut(df['imbd_votes'], bins=bins, labels=labels, right=False)
df = df.drop(columns="imbd_votes")

# Split The DataSet

In [208]:
# Data Processing
import pandas as pd
import numpy as np

# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image
from sklearn.neural_network import MLPClassifier

Choice of surveyed person

# Train & test the dataset

In [209]:
from sklearn.preprocessing import StandardScaler

In [210]:
from sklearn.neural_network import MLPRegressor

In [211]:
def calculate_average_human_ai_difference(xs, ys):
    AvgDifference = 0
    for (x,y) in zip(xs, ys):
        AvgDifference += abs(x-y)**3
    AvgDifference /= len(xs)
    return AvgDifference

def test_setups(dataFrame, numberOfIterations, probeSize, predictColumns, neuralNetworkParametersSets, getPredictions):
    savedTestPredSet = ([])
    globalParameterAverageDifferenceMeanArray = ([])
    globalColumnAverageDifferenceMeanArray = ([])
    
# - - - - - - - - - - - - Calculating for every paramether set - - - - - - - - - - - - 
    for neuralNetworkParametersSet in neuralNetworkParametersSets:
        parameterAverageDifferenceMeanArray = ([])
        # Get the data (ready for sgd and adam)
     #   activation, solver, alpha, learning_rate_init, hidden_layer_sizes, max_iter = neuralNetworkParametersSet
        
        #Mean of average difference for one parameter set
        parametersAverageDifferenceMean = 0

        # Mean of average differences for every one column saved
        columnAverageDifferenceMeanArray = ([])
       # temporaryDataFrame = uneditedDataFrame.copy()
    # - - - - - - - - - - - - Calculating for every user input - - - - - - - - - - - - 
        for predictColumn in predictColumns:
        #    temporaryDataFrame = uneditedDataFrame.copy()
            # Mean of average differences for one column
            columnAverageDifferenceMean = 0   
        # - - - - - - - - - - - - Calculating for every iteration - - - - - - - - - - - - 
            for iteration in range(0, numberOfIterations, 1):
                temporaryDataFrame = dataFrame.copy()


                # Drop every column with "rating_" that isnt the one to be predicted
                for col in list(temporaryDataFrame.columns):
                    if col.startswith("rating_") and col != predictColumn:
                        # Added: Check if column still exists before attempting to drop
                        if col in temporaryDataFrame.columns:
                            temporaryDataFrame.drop(columns=col, inplace=True)


                # Getting rid of unfilled data (to be used in the future)
                training_data = temporaryDataFrame[temporaryDataFrame[predictColumn] != -1]
                prediction_data = temporaryDataFrame[temporaryDataFrame[predictColumn] == -1]

                
                if temporaryDataFrame.empty:
                    print(f"Warning: temporaryDataFrame became empty after filtering for '{predictColumn}'. Skipping iteration.")
                    continue # Skip this iteration if no data remains


                # Splitting the dataset into train and test


                
              #  X = temporaryDataFrame.drop([predictColumn, 'name'], axis=1)
                #X_names = temporaryDataFrame['name']
               # y = temporaryDataFrame[predictColumn]
               # XTrain, XTest, yTrain, yTest = train_test_split(X, y, test_size=probeSize, random_state=42)

                XTrainNames = training_data['name']
                XTrain = training_data.drop(columns=[predictColumn, 'name'])
                yTrain = training_data[predictColumn]

               # XTest = toBePredicted.drop(predictColumn, axis=1)
                XTest = prediction_data.drop(columns=[predictColumn, 'name'])
                XTest_names = prediction_data['name']

                # Training the network
                clf = MLPRegressor(**neuralNetworkParametersSet)
              #  clf = MLPClassifier(activation = activation, solver=solver, alpha=alpha,
               #                     learning_rate_init = learning_rate_init,
                #                    hidden_layer_sizes=hidden_layer_sizes, max_iter = max_iter)
                
                x_scaler = StandardScaler()
                y_scaler = StandardScaler()
                X_train_scaled = x_scaler.fit_transform(XTrain)
                X_test_scaled = x_scaler.transform(XTest)
                y_train_scaled = y_scaler.fit_transform(yTrain.values.reshape(-1, 1))


                clf.fit(X_train_scaled, y_train_scaled.ravel())

                # Testing the network
                y_pred_scaled = clf.predict(X_test_scaled)
                yPred = y_scaler.inverse_transform(y_pred_scaled.reshape(-1, 1))

                #columnAverageDifferenceMean += calculate_average_human_ai_difference(yTest, yPred)
                if getPredictions:
                    print(f"DEBUG: Length of XTest_names: {len(XTest_names)}")
                    print(f"DEBUG: Length of yPred: {len(yPred)}")
                    # Store the input features (XTest) and the predictions (yPred)
                    savedTestPredSet = (XTest_names, yPred)
                    # This print is now for debugging and can be removed later
                    # print(yPred) dSet = (yTest, yPred)
            # Mean of average difference in current column prediction
            columnAverageDifferenceMean /= numberOfIterations
            # Saving such mean
            columnAverageDifferenceMeanArray.append(columnAverageDifferenceMean)

        # Mean of average difference in current parameter set
        parametersAverageDifferenceMean = np.mean(columnAverageDifferenceMeanArray)
        parameterAverageDifferenceMeanArray.append(parametersAverageDifferenceMean)
        globalColumnAverageDifferenceMeanArray.append(columnAverageDifferenceMeanArray)

    #globalParameterAverageDifferenceMeanArray.append(parametersAverageDifferenceMean)
       # print("Appending")
        globalParameterAverageDifferenceMeanArray.append(parameterAverageDifferenceMeanArray) # Use extend to flatten if parameterAverageDifferenceMeanArray is a list of single values
    return globalParameterAverageDifferenceMeanArray, globalColumnAverageDifferenceMeanArray, savedTestPredSet




In [212]:
import warnings
from sklearn.exceptions import ConvergenceWarning

# Ignore all ConvergenceWarnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)


In [216]:
import itertools

predictColumns = ("rating_kuby",)


activationSet = ("relu",)
solverSet = ("adam",)
alphaSet = (0.1, )
learning_rate = ("adaptive",)
learning_rate_initSet =  (0.1,)
hidden_layer_sizesSet = ((8,16, 32, 16, 8),)
max_iterSet = (100000, )

# Generate all combinations
neuralNetworkParametersSets = []
for activation, solver, alpha, learning_rate, learning_rate_init, hidden_layer_sizes, max_iter in itertools.product(
    activationSet,
    solverSet,
    alphaSet,
    learning_rate,
    learning_rate_initSet,
    hidden_layer_sizesSet,
    max_iterSet
):
    param_set = {
        "activation": activation,
        "solver": solver,
        "alpha": alpha,
        "learning_rate": learning_rate,
        "learning_rate_init": learning_rate_init,
        "hidden_layer_sizes": hidden_layer_sizes,
        "max_iter": max_iter
    }
    neuralNetworkParametersSets.append(param_set)

#print(f"DEBUG: Number of neuralNetworkParametersSets generated: {len(neuralNetworkParametersSets)}")
#print(f"DEBUG: Contents of neuralNetworkParametersSets: {neuralNetworkParametersSets}")

parameter, _, savedData = test_setups(dataFrame = df, numberOfIterations = 1,
                                              probeSize = 0.3, predictColumns=predictColumns,neuralNetworkParametersSets=neuralNetworkParametersSets,
                                              getPredictions=True)

chosenRating = predictColumns
if savedData:
    XTest_names_output, yPred_output = savedData
    
    yPred_output = yPred_output.flatten()
    
    # Create a new DataFrame with the names and predictions
    results_df = pd.DataFrame({
        "name": XTest_names_output.reset_index(drop=True),
        f"predicted_{predictColumns}": yPred_output
    })
    
    # --- Save the DataFrame to a new CSV file ---
    results_df.to_csv(f'predipocted_ratings{predictColumns}.csv', index=False)
    
    print(f"Predictions saved to predicted_ratings{predictColumns}.csv")
    print(results_df.head())
else:
    print("No predictions were saved.")
#print(parameter)
#print(globalTrainingSet)

DEBUG: Length of XTest_names: 201
DEBUG: Length of yPred: 201
Predictions saved to predicted_ratings('rating_kuby',).csv
                    name  predicted_('rating_kuby',)
0           12 Angry Men                    6.265747
1       12 Years a Slave                    7.560366
2                   1917                    6.751972
3  2001: A Space Odyssey                    8.524257
4               3 Idiots                    8.043756


In [214]:
import math

In [215]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from collections import Counter # Import Counter for easily counting occurrences
from matplotlib.colors import Normalize # For normalizing counts to colormap range
import numpy as np
import math

chosenRating = "rating_ali"
yTest, yPred = globalTrainingSet

# --- DEBUGGING AND FIX START ---
print(f"Shape of yTest before flattening: {yTest.shape}")
print(f"Shape of yPred before flattening: {yPred.shape}")

# Flatten the arrays to ensure they are 1D
yTest_flat = np.ravel(yTest)
yPred_flat = np.ravel(yPred)

# --- DEBUGGING AND FIX END ---

minimal = yTest_flat.min() if yTest_flat.min() < yPred_flat.min() else yPred_flat.min()
maximal = yTest_flat.max() if yTest_flat.max() > yPred_flat.max() else yPred_flat.max()

x = range(math.floor(minimal),math.ceil(maximal+1),1)
y = range(math.floor(minimal),math.ceil(maximal+1),1)

# Pass the flattened arrays directly to zip
data_points = list(zip(yTest_flat, yPred_flat))

point_counts = Counter(data_points)

unique_points = list(point_counts.keys())
counts = np.array(list(point_counts.values()))

min_count = counts.min()
max_count = counts.max()

#('viridis', 'plasma', 'hot', 'magma', 'cool')
cmap = cm.plasma

norm = Normalize(vmin=min_count, vmax=max_count if max_count > min_count else min_count + 1)


plt.figure(figsize=(10, 8))

for (test_val, pred_val) in unique_points:
    count = point_counts[(test_val, pred_val)]
    color = cmap(norm(count))
    plt.scatter(test_val, pred_val, color=color, s=50, alpha=0.7)

plt.plot(x, y, color='gray', alpha=0.6, linestyle='--', label='Idealne przewidywanie')

plt.xlabel("Odpowiedź człowieka")
plt.ylabel("Przewidziana odpowiedź przez algorytm")

sm = cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=plt.gca())
cbar.set_label('Liczba punktów o tej samej pozycji')


chosenRating_name = chosenRating.replace("rating_", "")
plt.title(f"Przewidywanie oceny ankietowanego ( {chosenRating_name} ) wybranych filmów z top250 filmów IMDb")
plt.legend()
plt.grid(True, linestyle=':', alpha=0.7)
plt.tight_layout()
plt.show()

ValueError: not enough values to unpack (expected 2, got 0)

# wybrany setup:
activationSet = ("relu",)

solverSet = ("adam",)

alphaSet = (0.1, )

learning_rate = ("adaptive",)

learning_rate_initSet =  (0.1,)

hidden_layer_sizesSet = ((8,16, 32, 16, 8),)

max_iterSet = (100000, )

In [ ]:
import itertools

predictColumns = ("rating_kuby", "rating_ali")


activationSet = ("relu",)
solverSet = ("adam",)
alphaSet = (0.1, )
learning_rate = ("adaptive",)
learning_rate_initSet =  (0.1,)
hidden_layer_sizesSet = ((8,16, 32, 16, 8),)
max_iterSet = (100000, )

# Generate all combinations
neuralNetworkParametersSets = []
for activation, solver, alpha, learning_rate, learning_rate_init, hidden_layer_sizes, max_iter in itertools.product(
    activationSet,
    solverSet,
    alphaSet,
    learning_rate,
    learning_rate_initSet,
    hidden_layer_sizesSet,
    max_iterSet
):
    param_set = {
        "activation": activation,
        "solver": solver,
        "alpha": alpha,
        "learning_rate": learning_rate,
        "learning_rate_init": learning_rate_init,
        "hidden_layer_sizes": hidden_layer_sizes,
        "max_iter": max_iter
    }
    neuralNetworkParametersSets.append(param_set)

print(f"DEBUG: Number of neuralNetworkParametersSets generated: {len(neuralNetworkParametersSets)}")
print(f"DEBUG: Contents of neuralNetworkParametersSets: {neuralNetworkParametersSets}")

parameter, _, globalTrainingSet = test_setups(dataFrame = df, numberOfIterations = 1,
                                              probeSize = 0.3, predictColumns=predictColumns,neuralNetworkParametersSets=neuralNetworkParametersSets,
                                              getPredictions=True)
print(parameter)
print(globalTrainingSet)